Case 6
Fabian Brock
i6248959

# LIBOR Market Model
We want to consider a LIBOR Market Model with forward LIBOR rates 𝑓𝑓𝑖𝑖(𝑡𝑡) with Δ𝑇𝑇 = 1 and 𝑁𝑁 = 10.
So, the final time-point in this model is 𝑇𝑇10 = 10 years. The initial term-structure of interest rates is
equal to 𝑓𝑓𝑖𝑖(0) = 0.03 = 3% for all 𝑖𝑖 = 0 ... 9. We will model all LIBOR rates as Gaussian martingales:
𝑓𝑓𝑖𝑖(𝑡𝑡) = 𝜎𝜎𝑖𝑖 𝑑𝑑𝑊𝑊 ℚ𝑖𝑖+1
(𝑡𝑡), with 𝜎𝜎𝑖𝑖 ≡ 0.01 = 1% for all 𝑖𝑖 = 0 ... 9.

## Discount bond prices

In [117]:
import numpy
import numpy as np
from scipy.stats import norm
import pandas as pd

In [118]:
annual_interest_rate = 0.03  # 3% interest rate
maturities = range(1, 11)  # Maturities from 1 to 9 years

# using the formula P(0, T) = 1 / (1 + r)^T
bond_prices = {T: 1 / ((1 + annual_interest_rate) ** T) for T in maturities}
bond_prices

{1: 0.970873786407767,
 2: 0.9425959091337544,
 3: 0.9151416593531596,
 4: 0.8884870479156888,
 5: 0.8626087843841639,
 6: 0.8374842566836542,
 7: 0.8130915113433536,
 8: 0.7894092343139355,
 9: 0.7664167323436267,
 10: 0.7440939148967249}

In [119]:
initial_forward_rate = 0.03
sigma = 0.01
num_maturities = 9  # f0 to f9
num_paths = 10000
time_step = 1 # deltaT
total_time = 10  # sim time

# timepoints
num_steps = int(total_time / time_step)
time_points = np.linspace(0, total_time, num_steps + 1)

# init forward rates matrix
forward_rates = np.zeros((num_paths, num_maturities, num_steps + 1))
forward_rates[:, :, 0] = initial_forward_rate

# MC
np.random.seed(0)
for path in range(num_paths):
    for step in range(num_steps):
        dt = time_points[step + 1] - time_points[step]
        dW = np.random.normal(0, np.sqrt(dt), num_maturities)
        forward_rates[path, :, step + 1] = forward_rates[path, :, step] + sigma * dW

# avg forward rates of all paths
average_forward_rates = forward_rates.mean(axis=0)

# avg forward rates at year end
average_forward_rates_at_year_end = average_forward_rates[:, ::int(1/time_step)]
# avg per column
average_forward_rates = average_forward_rates_at_year_end.mean(axis=0)
average_forward_rates

array([0.03      , 0.02997317, 0.02996082, 0.02995905, 0.03002102,
       0.03000746, 0.03009609, 0.03009348, 0.03009618, 0.03012247,
       0.03017021])

In [120]:
# Calculating the Monte Carlo values for discount bonds

# Initialize an array to store the Monte Carlo values of the bonds
mc_bond_values = np.zeros((num_maturities, num_steps + 1))


# Calculate the present value of each bond for each path and then average
for i in range(num_maturities):
    for path in range(num_paths):
        # Calculate the discount factor for each path at maturity T_i
        discount_factor = np.prod(1 + time_step * forward_rates[path, :i+1, i])
        mc_bond_values[i] += discount_factor

    # Average the discount factors over all paths
    mc_bond_values[i] /= num_paths

# Monte Carlo values of the bonds at t=0
mc_values_at_t0 = 1 / mc_bond_values[:, 0]
mc_values_at_t0

array([0.97087379, 0.94260751, 0.91518797, 0.88865491, 0.8622649 ,
       0.83714808, 0.81246462, 0.78884481, 0.76582021])

In [121]:
# Adjusted parameters for the Monte Carlo simulation using D_10(t) as the numeraire
num_maturities = 10  # Number of forward rates (f0 to f9, inclusive)
num_steps = num_maturities  # Total simulation time in years
time_step = 1  # Time step in years, Δ = 1
sigma = 0.01  # Constant volatility 1%

# Initializing matrix to store simulated forward rates for each path
forward_rates_mc = np.zeros((num_paths, num_maturities, num_steps + 1))
forward_rates_mc[:, :, 0] = initial_forward_rate

# Monte Carlo simulation under Q_10 measure
np.random.seed(0)  # For reproducibility
for path in range(num_paths):
    for step in range(num_steps):
        dW = np.random.normal(0, np.sqrt(time_step), num_maturities)  # Brownian motion increment
        forward_rates_mc[path, :, step + 1] = forward_rates_mc[path, :, step] + sigma * dW

# Calculating the Monte Carlo values for discount bonds under Q_10
mc_discount_bond_values = np.zeros((num_maturities, num_steps + 1))

# Calculating the present value of each bond for each path and then averaging
for i in range(num_maturities):
    for path in range(num_paths):
        # Calculate the discount factor for each path at maturity T_i
        discount_factor = np.prod(1 / (1 + time_step * forward_rates_mc[path, :i+1, i]))
        mc_discount_bond_values[i] += discount_factor

    # Average the discount factors over all paths
    mc_discount_bond_values[i] /= num_paths

# Extracting the Monte Carlo values of the bonds at t=0
mc_discount_values_at_t0 = mc_discount_bond_values[:, 0]
mc_discount_values_at_t0



array([0.97087379, 0.94283202, 0.91563443, 0.88968214, 0.86440704,
       0.83971669, 0.81588532, 0.79332587, 0.77152657, 0.74940902])

In [122]:
# convert bond prices values to numpy array
libor = np.array(list(bond_prices.values()))
both = pd.DataFrame(np.vstack([libor, mc_discount_values_at_t0]).T, columns=["LIBOR", "MC"])
both["Difference"] = both.LIBOR - both.MC
both

,LIBOR,MC,Difference
0,0.970874,0.970874,8.570922e-14
1,0.942596,0.942832,-2.361074e-04
2,0.915142,0.915634,-4.927686e-04
3,0.888487,0.889682,-1.195094e-03
4,0.862609,0.864407,-1.798257e-03
5,0.837484,0.839717,-2.232433e-03
6,0.813092,0.815885,-2.793813e-03
7,0.789409,0.793326,-3.916634e-03
8,0.766417,0.771527,-5.109835e-03
9,0.744094,0.749409,-5.315102e-03


In [123]:
# Parameters
initial_forward_rate = 0.03  # 3%
sigma = 0.01  # Volatility 1%
num_maturities = 10  # Including T_0 to T_9
num_paths = 10000
total_time = 10  # Total time T_10
time_step = 1  # Delta T
num_steps = total_time // time_step  # Number of time steps

# Initialize forward rates matrix
forward_rates = np.full((num_paths, num_maturities, num_steps + 1), initial_forward_rate)

# Calculate drift corrections
# Note: This is a simplified approach; actual drift terms depend on your specific LMM setup
drift_corrections = np.zeros((num_maturities, num_steps))


# Monte Carlo simulation
np.random.seed(0)
for path in range(num_paths):
    for step in range(num_steps):
        dt = time_step
        dW = np.random.normal(0, np.sqrt(dt), num_maturities)
        drift = drift_corrections[:, step]
        forward_rates[path, :, step + 1] = forward_rates[path, :, step] * np.exp(drift * dt + sigma * dW)

# Calculate discount bond prices from forward rates
# Note: This requires a specific method to convert forward rates to bond prices
# Here we use a placeholder method
def forward_rates_to_bond_prices(forward_rates):
    # Placeholder for the actual conversion from forward rates to bond prices
    return np.exp(-forward_rates)

# Calculate bond prices
bond_prices_mc = forward_rates_to_bond_prices(forward_rates.mean(axis=0))

# Compare with initial term-structure (flat at 3%)
initial_bond_prices = 1 / ((1 + initial_forward_rate) ** np.arange(1, num_maturities + 1))
comparison = np.vstack([initial_bond_prices, bond_prices_mc.mean(axis=1)]).T

print("Initial vs MC Bond Prices:")
print(comparison)


Initial vs MC Bond Prices:
[[0.97087379 0.97043955]
 [0.94259591 0.97043454]
 [0.91514166 0.97043831]
 [0.88848705 0.9704408 ]
 [0.86260878 0.9704407 ]
 [0.83748426 0.97043418]
 [0.81309151 0.97043522]
 [0.78940923 0.97043989]
 [0.76641673 0.97043676]
 [0.74409391 0.97042399]]


## Gaussian Swaption Formulas

In [124]:
def gauss_payer_swaption(principal, strike, volatility, option_maturity, current_time, discount_factors):

    sum_discount_factors = sum(discount_factors)
    forward_rate = (discount_factors[0] - discount_factors[-1]) / sum_discount_factors
    d = (forward_rate - strike) / (volatility * np.sqrt(option_maturity - current_time))
    phi_d = np.exp(-0.5 * d**2) / np.sqrt(2 * np.pi)

    swaption_price = principal * sum_discount_factors * ((forward_rate - strike) * norm.cdf(d) + volatility * np.sqrt(option_maturity - current_time) * phi_d)
    return swaption_price

# Parameters
principal = 1
volatility = 0.01  # gaussian
option_maturity = 10
current_time = 0
maturities = np.arange(1, 10)  # maturieites from 1 to 9
strike_rates = np.arange(0.01, 0.06, 0.01)  # strike rates from 1% to 5%
annual_interest_rate = 0.03  # term strucuture 3%

# "discount facotrs"
#discount_factors = [np.exp(-annual_interest_rate * t) for t in range(1, option_maturity + 1)]
discount_factors = [ 1 / ((1 + annual_interest_rate) ** T) for T in maturities]


# swaption prices
swaption_prices = np.array([
    [gauss_payer_swaption(principal, strike, volatility, option_maturity, current_time, discount_factors[:maturity])
     for strike in strike_rates] for maturity in maturities])


# make dataframe out of swaption prices
swaption_prices_df = pd.DataFrame(swaption_prices, index=maturities, columns=strike_rates)
swaption_prices_df


,0.01,0.02,0.03,0.04,0.05
1,0.008001,0.004911,0.002818,0.001505,0.000746
2,0.028986,0.019472,0.012320,0.007305,0.004040
3,0.051075,0.035266,0.022997,0.014087,0.008067
4,0.072929,0.051026,0.033763,0.021012,0.012239
5,0.094303,0.066494,0.044377,0.027878,0.016402
6,0.115130,0.081593,0.054764,0.034616,0.020502
7,0.135392,0.096299,0.064893,0.041199,0.024517
8,0.155090,0.110605,0.074756,0.047616,0.028435
9,0.174230,0.124512,0.084350,0.053863,0.032254
